In [1]:
# Advent of Code 2022
# Day 15
# https://adventofcode.com/2022/day/15

import re

with open('input.txt') as f:
    data = f.read()
    lines = data.split('\n')
    lines = list(filter(None, lines))



In [2]:
# parse strings to get sensor and beacon coordinates
def get_coords(line):
    re_sensors_beacons = re.compile(r'(\-*\d+)')
    xy_nums = list(re_sensors_beacons.findall(line))
    for i, num in enumerate(xy_nums):
        xy_nums[i] = int(num)
    return (xy_nums)



In [3]:
# # set of all beacons
# beacon_points = set()
# for l in lines:
#     coords = get_coords(l)
#     x = coords[2]
#     y = coords[3]
#     coords = (x, y)
#     beacon_points.add(coords)

# set of all beacons in a given row
def get_beacon_points_row(test_row):
    beacon_points_row = set()
    for l in lines:
        coords = get_coords(l)
        x = coords[2]
        y = coords[3]
        if y == test_row:
            coords = (x, y)
            beacon_points_row.add(coords)
    return beacon_points_row


# # set of all sensors
# sensor_points = set()
# for l in lines:
#     coords = get_coords(l)
#     x = coords[0]
#     y = coords[1]
#     coords = (x, y)
#     sensor_points.add(coords)

# set of all sensors in a given row
def get_sensor_points_row(test_row):
    sensor_points_row = set()
    for l in lines:
        coords = get_coords(l)
        x = coords[0]
        y = coords[1]
        if y == test_row:
            coords = (x, y)
            sensor_points_row.add(coords)
    return sensor_points_row


get_beacon_points_row(3)


set()

In [4]:
def get_dist (line):
    sx, sy, bx, by = get_coords(line)
    dist = abs(sx-bx) + abs(sy-by)
    return dist

In [5]:
# list all points within range of a sensor-beacon pair
# but only if they are also within range of a given row
def get_nearby_points(line, test_row):

    # generate a set to contain all beaconless points
    nearby = set()

    # identify sensor and beacon x-y coordinates
    sx, sy, bx, by = get_coords(line)


    # get distance between beacon and sensor
    dist = get_dist(line)

    # test to see if the area covered by this
    # sensor-beacon pair is within range of the
    # test line
    if abs(sy-test_row) <= dist:

        # # for each x value, caluclate min and max y values
        # for x in range(sx-dist, sx+dist+1):
        #     x_dist = abs(x-sx)
        #     y_dist = dist - x_dist
        #     y_min = sy - y_dist
        #     y_max = sy + y_dist

        #     # add all points between min and max x-y coords to the set
        #     points_list = [(x,y) for y in range(y_min, y_max+1)]
        #     nearby.update(points_list)

        y_dist = abs(sy - test_row)
        x_dist = dist - y_dist
        x_min = sx - x_dist
        x_max = sx + x_dist
        points_list = [(x, test_row) for x in range(x_min, x_max+1)]
        nearby.update(points_list)
    
    return nearby




In [6]:
# get all of the points within range of all beacon-sensor pairs near
# the test row

def get_all_points_near(test_row):
    
    all_nearby_points = set()
    for l in lines:
        all_nearby_points.update(get_nearby_points(l, test_row))

    # eliminate points where beacons or sensors are already known to exist:
    beaconless = all_nearby_points - get_beacon_points_row(test_row) - get_sensor_points_row(test_row)

    return beaconless



In [7]:
# Count beaconless points for a given row:
def count_nearby(test_row):

    beaconless = get_all_points_near(test_row)

    nearby_row_list = [i for i in beaconless if i[1]==test_row]
    return len(nearby_row_list)

answer = count_nearby(2000000)

print(answer)



4560025


In [8]:
# Part 2 New Strategy:

# based on hints from others:  check all points around every sensor instad of iterating 
# through rows.

#same as part 1
with open('input.txt') as f:
    data = f.read()
    lines = data.split('\n')
    lines = list(filter(None, lines))


In [9]:
def get_sensor_info(line):
    """get sensor coordinates and range (distance from
    sensor to beacon):  (sx, sy, range)"""
    re_sensors_beacons = re.compile(r'(\-*\d+)')
    xy_nums = list(re_sensors_beacons.findall(line))
    for i, num in enumerate(xy_nums):
        xy_nums[i] = int(num)
    sx = xy_nums[0]
    sy = xy_nums[1]
    bx = xy_nums[2]
    by = xy_nums[3]
    s_point = (sx, sy)
    s_range = abs(sx-bx) + abs(sy-by)
    # s_zero_dist = abs(sx) + abs(sy)

    return(sx, sy, s_range)



In [10]:

# find min/max x value for each row's checked range using each sensor.
# sort the ranges by x value so you can review them for overlaps / adjacency in order.
# if max (range) >= min (next_range)+1:
#       gap exists. Beacon found.


sensors = []
for l in lines:
    sensors.append(get_sensor_info(l))

def get_sensor_range_for_row(sensor, row):

    sx = sensor[0]
    sy = sensor[1]
    s_range = sensor[2]

    if abs(sy-row) <= s_range:

        y_dist = abs(sy-row)
        x_dist = s_range - y_dist

        x_min = sx - x_dist
        x_max = sx + x_dist

        r_range = (x_min, x_max)
        return r_range
        


In [11]:
def get_all_ranges_for_row(row):
    row_ranges = []
    for s in sensors:
        s_r_range = get_sensor_range_for_row(s,row)
        row_ranges.append(s_r_range)
        row_ranges = [r for r in row_ranges if r != None]
        row_ranges.sort()
    
    return row_ranges
    


In [12]:
def find_gaps_in_ranges(sorted_row_ranges, max_xy):

    x_min = None
    x_max = None

    for r in sorted_row_ranges:
        # print(f'\n{r}')
        r_min = r[0]
        r_max = r[1]
        if x_min == None:
            x_min = r_min
            # print(f'first if: X_min was none, now it is {x_min}')
        if x_max == None:
            x_max = r_max
            # print(f'first if: X_max was none, now it is {x_max}')

        # print(f'x min: {x_min}, x max: {x_max}')

        # if 0 is outside the min/max range, column 0 is a gap.
        if x_min > 0:
            return 0

        # if the new range's min is still within the max_xy range:
        if r_min <= max_xy:

            # if ranges don't overlap:
            # min is higher than previous max +1
            if r_min > x_max+1:
                # print(f'second if: r_min is {r_min}')
                # print(f'second if: x_max is {x_max}')
                x_gap = x_max+1
                return x_gap
                    
            if r_max > x_max:
                x_max = r_max  


In [13]:
def find_gap(max_xy):
    for y in range(max_xy+1):

        sorted_row_ranges = get_all_ranges_for_row(y)

        for s in sorted_row_ranges:
            gap_x = find_gaps_in_ranges(sorted_row_ranges, max_xy)
        
        if gap_x != None:
            return (gap_x, y)



In [14]:
# find the tuning frequency

gap = find_gap(4000000)

x = gap[0]
y = gap[1]

tuning_frequency = 4000000 *  x + y

print(tuning_frequency)


12480406634249
